# Generate object(sign) and annotations on draw

In [85]:
%matplotlib inline
import os
from glob import glob
from matplotlib import pyplot
from PIL import Image
import numpy as np

import xml.etree.ElementTree as ET
import random


In [87]:
draw_data = glob('./data/backGround/draw/*.png')
annotaions_data = glob('./data/backGround/annotations/*.xml')
sign_data = glob('./data/backGround/sign/*.png')


size = 15, 15

for s in sign_data:
    im = Image.open(s)
    im.thumbnail(size, Image.ANTIALIAS)
    im.save(s)

In [83]:
annotaions_data

['./data/backGround/annotations/b2.xml',
 './data/backGround/annotations/3f.xml',
 './data/backGround/annotations/b3.xml',
 './data/backGround/annotations/8f.xml',
 './data/backGround/annotations/b1.xml',
 './data/backGround/annotations/2f.xml',
 './data/backGround/annotations/1f.xml',
 './data/backGround/annotations/b4.xml',
 './data/backGround/annotations/4f.xml',
 './data/backGround/annotations/11f.xml',
 './data/backGround/annotations/12f.xml',
 './data/backGround/annotations/9f.xml',
 './data/backGround/annotations/6f.xml',
 './data/backGround/annotations/10f.xml',
 './data/backGround/annotations/7f.xml',
 './data/backGround/annotations/5f.xml']

In [84]:
draw_data 

['./data/backGround/draw/7f.png',
 './data/backGround/draw/8f.png',
 './data/backGround/draw/12f.png',
 './data/backGround/draw/5f.png',
 './data/backGround/draw/b4.png',
 './data/backGround/draw/b3.png',
 './data/backGround/draw/9f.png',
 './data/backGround/draw/2f.png',
 './data/backGround/draw/3f.png',
 './data/backGround/draw/b2.png',
 './data/backGround/draw/1f.png',
 './data/backGround/draw/6f.png',
 './data/backGround/draw/4f.png',
 './data/backGround/draw/10f.png',
 './data/backGround/draw/11f.png',
 './data/backGround/draw/b1.png']

In [ ]:
import xml.etree.cElementTree as ET


def makeXml(dr_fileName):

    root = ET.Element("root")
    doc = ET.SubElement(root, "doc")

    ET.SubElement(doc, "field1", name="blah").text = "some value1"
    ET.SubElement(doc, "field2", name="asdfasd").text = "some vlaue2"

    tree = ET.ElementTree(root)
    tree.write("filename.xml")




def makeSign(clss, amount, dr_fileName):
    tree = ET.parse(dr_fileName)
    root = tree.getroot()
    width = root[4][0].text
    height = root[4][1].text
    depth = root[4][2].text
                     
    for k in amount:    
        xmin = random.sample(range(100,width))
        ymin = random.sample(range(100,height))
                
        xmax = xmin + 16
        ymin = ymin + 16
                
        s_img = cv2.imread("./data/backGround/sign/"+clss+'.png', -1)
        s_img.copyTo(dr_fileName(cv::Rect(xmin,ymin,s_img.cols, s_img.rows)));
        im = Image.open(i)
            
    
    

In [88]:


for i in draw_data:
    dr_fileName = i.split('draw/')[1]
    dr_fileName = dr_fileName.split('.png')[0]    
    for j in annotaions_data:
        ann_fileName = j.split('annotations/')[1]
        ann_fileName = ann_fileName.split('.xml')[0]
        
        if dr_fileName == ann_fileName:

            


            
            


SyntaxError: invalid syntax (<ipython-input-88-c5912b77f688>, line 1)

In [76]:
tree = ET.parse(annotaions_data[0])
root = tree.getroot()
root[0].text

In [ ]:
count = 0

for clss in root.iter('name'):
    count+=1
    if clss.text == ''
    print(clss.text)

print(count)